In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid
import matplotlib.pyplot as plt
import numpy as np
import os
import json
from PMLutils import *
torch.cuda.is_available()

# A1

In [ ]:
cuda = torch.cuda.is_available()
batch_size = 128
num_epochs = 10

torch.manual_seed(1) # args.seed

device = torch.device("cuda" if cuda else "cpu") # args.cuda
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {} # args.cuda

# Get train and test data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=1, shuffle=True, **kwargs)



In [ ]:
from models.vae import VAE
from models.convVAE import ConvVAE
z = 2
model = VAE(z).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(f"trainable parameters: {params:,}")
prev_loss = 1e9
for epoch in range(1, num_epochs + 1):
    train_VAE(model, optimizer, train_loader, device)
    loss = test_VAE(model, device, test_loader)
    with torch.no_grad():
        sample = torch.randn(64, z).to(device) # 20 -> 2
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/VAE/sample_' + str(epoch) + '.png')
    if loss < prev_loss:
        torch.save(model.state_dict(), "weights/VAE.pth")
        prev_loss = loss

In [ ]:
z = 2
model = ConvVAE(out_channels = 64, kernel_size = 4, hiddenDim=z).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(f"trainable parameters: {params:,}")

In [ ]:
prev_loss = 1e9
for epoch in range(1, num_epochs + 1):
    train_VAE(model, optimizer, train_loader, device)
    loss = test_VAE(model, device, test_loader)
    with torch.no_grad():
        sample = torch.randn(64, z).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/convVAE/sample_' + str(epoch) + '.png')
    if loss < prev_loss:
        torch.save(model.state_dict(), "weights/convVAE.pth")
        prev_loss = loss

# A2


Diffusion model

perhaps need to renormalize images for it to work: transforms.Normalize((0.5,),(1.0))

In [ ]:
from models.diffusion import DenoisingDiffusion, DeNoise, DeNoiseBIG

tf = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0))])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=tf),
    batch_size=batch_size, shuffle=True, **kwargs)

mps = torch.backends.mps.is_available()

torch.manual_seed(1) # args.seed
if mps:
    device = torch.device("mps")
else:
    device = torch.device("cuda" if cuda else "cpu") # args.cuda
print("Using device ", device)

eps_model = DeNoiseBIG()
eps_model.to(device)
diff = DenoisingDiffusion(eps_model, device=device) 
optimizer = torch.optim.Adam(eps_model.parameters(), lr=1e-3)
prev_loss = 1e9
for epoch in range(1, num_epochs):
    tqdm_ = tqdm(train_loader)
    for data, _ in tqdm_:
        diff.train() # so that everything has gradients and we can do backprop and so on...
        data = data.to(device)
        optimizer.zero_grad() # "reset" gradients to 0 for text iteration
        loss = diff(data)
        loss.backward() # calc gradients
        optimizer.step() # backpropagation
        tqdm_.set_description(f"loss: {loss:.4f}")
    
    diff.eval()
    with torch.no_grad():
        sample = diff.sample_p(n=64)
        save_image(sample.view(64, 1, 28, 28).cpu(),
                    'results/diffusion/sample_' + str(epoch) + '.png')
    if loss < prev_loss:
        torch.save(diff.state_dict(), "weights/diffusion.pth")
        prev_loss = loss

PPCA

In [ ]:
from models.ppca import PPCA

ppca = PPCA(train_loader.dataset, 4)
samples = ppca.sample()


Bayesian VAE (failed)

In [ ]:
from models.bayesian import BayesianVAE


prior = torch.distributions.Normal(torch.tensor([0]), torch.tensor([1]))
model = BayesianVAE(prior)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
reconstrction_loss = nn.CrossEntropyLoss()

model.train() # so that everything has gradients and we can do backprop and so on...
train_loss = 0
for batch_idx, (data, _) in enumerate(train_loader):
    data = data.to(device)
    optimizer.zero_grad() # "reset" gradients to 0 for text iteration
    recon_batch, mu, logvar = model(data)
    
    loss = model.loss() + reconstrction_loss(recon_batch, data)
    loss.backward() # calc gradients
    train_loss += loss.item()
    optimizer.step() # backpropagation

print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))



model.eval()
test_loss = 0
with torch.no_grad(): # no_grad turns of gradients...
    for i, (data, _) in enumerate(test_loader):
        data = data.to(device)
        recon_batch, mu, logvar = model(data)
        test_loss += model.KL_loss().item()

test_loss /= len(test_loader.dataset)
print('====> Test set loss: {:.4f}'.format(test_loss))

Metrics

In [ ]:
def produce_samples_vae(model, input, num_imgs, weights = None, device='cpu'):
   
    if weights is not None:
        model.load_state_dict(torch.load(weights, map_location=torch.device(device)))
    with torch.no_grad():
        samples = model.decode(input).cpu()
    return samples.unsqueeze(1)

real = []
for i,_ in test_loader: #Get the normalized images
    real.append(i) 
num_imgs = len(real)
real = torch.stack(real).squeeze(1)
  

In [ ]:

from models.convVAE import ConvVAE
from models.vae import VAE
from models.diffusion import DenoisingDiffusion, DeNoise
from torchmetrics.image.inception import InceptionScore

make_new = False
#Record results
models = [(
            VAE(), "weights/VAE.pth", (1,2)), (
            ConvVAE(out_channels = 64, kernel_size = 4, hiddenDim=2), "weights/convVAE.pth", (1,2)), (
            DenoisingDiffusion(DeNoise()), "weights/diffusion.pth", (1, 1,28,28))
            ]


if make_new:
    with open("results/scores.txt", "w") as file:
        file.write("Optimal scores:\n")
        file.write(json.dumps(test_suite(real, real)))
        file.write("\n------------\n")

for m,w in [(VAE(), "weights/VAE.pth"), (ConvVAE(out_channels = 64, kernel_size = 4, hiddenDim=2), "weights/convVAE.pth")]:
    fake = produce_samples_vae(m, torch.randn(10_000,2), num_imgs=num_imgs, weights=w)
    scores_dict = test_suite(real, fake)
    with open("results/scores.txt", "a") as file:
        file.write(w.split("/")[-1][:-4] + " = ")
        file.write(json.dumps(scores_dict))
        file.write("\n")







    

In [ ]:
ks=[]; vs=[]
samples = [s/torch.max(s) for i,s in ppca.compute_pca_loop()]*1000
fake = torch.stack(samples).type(torch.float32)
print(fake.shape)
scores_dict = test_suite(real.cpu(), fake.cpu())
   
#scores_dict = {k:np.average(v) for k,v in zip(ks, vs)}
with open("results/scores.txt", "a") as file:
    file.write("\nPPCA = ")
    file.write(json.dumps(scores_dict))
    file.write("\n")

In [ ]:
eps_model = DeNoiseBIG()
eps_model.to(device)
model = DenoisingDiffusion(eps_model, device=device)
model.load_state_dict(torch.load("weights/diffusion.pth", map_location=torch.device(device)))
with torch.no_grad():
    samples = []
    for i in range(1,101):
        sample = model.sample_p(100)
        print(f"{i}/{100}")
        m = torch.max(sample)
        samples.append(sample/m)
        
    fake = torch.stack(samples, dim=0).squeeze(1)
    
    scores_dict = test_suite(real.cpu(), fake.cpu())
    with open("results/scores.txt", "a") as file:
        file.write("\nDiffusion = ")
        file.write(json.dumps(scores_dict))
        file.write("\n")

In [ ]:
js = []
from PIL import Image
path = "results/PPCA/"
for img in os.listdir(path):

    j = Image.open(path+img)
    conv = transforms.ToTensor()
    j = conv(j)
   
    js.append(j)
    print(j.shape)

js = torch.stack(js)
js = torch.roll(js, 1)
print(js.shape)

img = make_grid(js, nrow=4)

print(img.shape)
save_image(img, "results/PPCA/ppca.png")